<a href="https://colab.research.google.com/github/tayyabnazirofficial/Demo/blob/main/Colab%20RDP/Colab%20RDP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Colabcraft** : Stream Minecraft Java (TLauncher) to Chromebook using Chrome RDP

> **Warning : Not for Cryptocurrency Mining<br></br>**

As this runs TLauncher, be warned that you won't be able to join non-cracked servers. This includes Hypixel.<br>
Technically, the code <b>could</b> be changed to use the official Minecraft client, but in this day and age it's unsafe to type in your MC account details on anywhere you don't trust.

**[Colab Hacks](https://github.com/PradyumnaKrishna/Colab-Hacks)**

In [1]:
#@title **Create User**
#@markdown Just press the play button, no need to change the default variables.

username = "user" #@param {type:"string"}
password = "root" #@param {type:"string"}

print("Creating User and Setting it up")

# Creation of user
! sudo useradd -m $username &> /dev/null

# Add user to sudo group
! sudo adduser $username sudo &> /dev/null

# Set password of user to 'root'
! echo '$username:$password' | sudo chpasswd

# Change default shell from sh to bash
! sed -i 's/\/bin\/sh/\/bin\/bash/g' /etc/passwd

print("User Created and Configured")

Creating User and Setting it up
chpasswd: (user $username) pam_chauthtok() failed, error:
Authentication token manipulation error
chpasswd: (line 1, user $username) password not changed
User Created and Configured


In [ ]:
#@title **RDP**
#@markdown  It takes 4-5 minutes for installation

import os
import subprocess

#@markdown  Visit http://remotedesktop.google.com/headless and Copy the command after authentication

CRP = "" #@param {type:"string"}

#@markdown Enter a pin more or equal to 6 digits
Pin = 123456 #@param {type: "integer"}


class CRD:
    def __init__(self):
        os.system("apt update")
        self.installCRD()
        self.installDesktopEnvironment()
        self.installGoogleChorme()
        self.installTLauncher()
        self.finish()

    @staticmethod
    def installCRD():
        print("Installing Chrome Remote Desktop")
        subprocess.run(['wget', 'https://dl.google.com/linux/direct/chrome-remote-desktop_current_amd64.deb'], stdout=subprocess.PIPE)
        subprocess.run(['dpkg', '--install', 'chrome-remote-desktop_current_amd64.deb'], stdout=subprocess.PIPE)
        subprocess.run(['apt', 'install', '--assume-yes', '--fix-broken'], stdout=subprocess.PIPE)

    @staticmethod
    def installDesktopEnvironment():
        print("Installing Desktop Environment")
        os.system("export DEBIAN_FRONTEND=noninteractive")
        os.system("apt install --assume-yes xfce4 desktop-base xfce4-terminal")
        os.system("bash -c 'echo \"exec /etc/X11/Xsession /usr/bin/xfce4-session\" > /etc/chrome-remote-desktop-session'")
        os.system("apt remove --assume-yes gnome-terminal")
        os.system("apt install --assume-yes xscreensaver")
        os.system("systemctl disable lightdm.service")

    @staticmethod
    def installGoogleChorme():
        print("Installing Google Chrome")
        subprocess.run(["wget", "https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb"], stdout=subprocess.PIPE)
        subprocess.run(["dpkg", "--install", "google-chrome-stable_current_amd64.deb"], stdout=subprocess.PIPE)
        subprocess.run(['apt', 'install', '--assume-yes', '--fix-broken'], stdout=subprocess.PIPE)

    @staticmethod
    def installTLauncher():
        print("Installing TLauncher")
        subprocess.run(["wget", "https://github.com/RikyIsola/tlauncher-linux/releases/download/v1.0/tlauncher_1.0_amd64.deb"], stdout=subprocess.PIPE)
        subprocess.run(["dpkg", "--install", "tlauncher_1.0_amd64.deb"], stdout=subprocess.PIPE)
        subprocess.run(['apt', 'install', '--assume-yes', '--fix-broken'], stdout=subprocess.PIPE)

    @staticmethod
    def finish():
        print("Finalizing")
        os.system(f"adduser {username} chrome-remote-desktop")
        command = f"{CRP} --pin={Pin}"
        os.system(f"su - {username} -c '{command}'")
        os.system("service chrome-remote-desktop start")
        print("Finished Succesfully")


try:
    if username:
        if CRP == "":
            print("Please enter authcode from the given link")
        elif len(str(Pin)) < 6:
            print("Enter a pin more or equal to 6 digits")
        else:
            CRD()
except NameError as e:
    print("username variable not found")
    print("Create a User First")

In [ ]:
#@title **Automatically apply performance settings**
#@markdown Run to apply the premade performance settings for Optifine instead of changing them manually.<br>
#@markdown These premade settings may make the graphics look pretty terrible, but overall make the game more playable and less laggy.<br>
#@markdown In most cases you probably could just use the default settings, but if the lag becomes unbearable then you can use these instead.
!pip install gdown -q
!mkdir /home/user/.minecraft
!cd /home/user/.minecraft
!gdown https://drive.google.com/uc?id=1jWEH71TjAY-F3D-5q2atSl54fLhSJGix -O /home/user/.minecraft/options.txt -q
!gdown https://drive.google.com/uc?id=1GSEYPFqukw6MisUUiLyzjOL5JS1LXIRc -O /home/user/.minecraft/optionsof.txt -q
! runuser -l $user -c "sudo chmod u+rwx,go+rwx /home/user/.minecraft"  > /dev/null 2>&1


In [ ]:
#@title **Google Drive Mount**
#@markdown Google Drive used for files.<br>
#@markdown In this case, used for storing MC world saves.<br>
#@markdown Mounted at `user` Home directory inside drive folder<br>
#@markdown Run <b>before</b> using options below!

def MountGDrive():
    from google.colab import drive

    ! runuser -l $user -c "yes | python3 -m pip install --user google-colab"  > /dev/null 2>&1

    mount = """from os import environ as env
from google.colab import drive

env['CLOUDSDK_CONFIG']  = '/content/.config'
drive.mount('{}')""".format(mountpoint)

    with open('/content/mount.py', 'w') as script:
        script.write(mount)

    ! runuser -l $user -c "python3 /content/mount.py"

try:
    if username:
        mountpoint = "/home/"+username+"/drive"
        user = username
except NameError:
    print("username variable not found, mounting at `/content/drive' using `root'")
    mountpoint = '/content/drive'
    user = 'root'

MountGDrive()

In [ ]:
#@title **RESTORE MC Saves from Drive to RDP**
#@markdown Use this to copy your Minecraft worlds from your Google Drive to your RDP server.<br>
#@markdown <b>EXTRA TIP!</b> You can add custom worlds by uploading them to the "0_MC_Saves" folder in your Google Drive!
! runuser -l $user -c "mkdir -p /home/user/.minecraft/saves"  > /dev/null 2>&1
! runuser -l $user -c "mkdir -p /home/user/drive/MyDrive/0_MC_Saves"  > /dev/null 2>&1
! runuser -l $user -c "cp -r /home/user/drive/MyDrive/0_MC_Saves/saves /home/user/.minecraft/"  > /dev/null 2>&1
print("Done! If it didn't work, it's likely you don't have any worlds backed up.")

In [ ]:
#@title **BACKUP MC Saves from Drive to RDP**
#@markdown Use this to copy your Minecraft worlds from your RDP server to your Google Drive.<br>
! runuser -l $user -c "mkdir -p /home/user/.minecraft/saves"  > /dev/null 2>&1
! runuser -l $user -c "mkdir -p /home/user/drive/MyDrive/0_MC_Saves"  > /dev/null 2>&1
! runuser -l $user -c "cp -r /home/user/.minecraft/saves /home/user/drive/MyDrive/0_MC_Saves/"  > /dev/null 2>&1
print("Done! If it didn't work, it's likely you don't have any worlds.")

In [ ]:
#@title **Anti-Shutdown**
#@markdown Run this when you're playing Minecraft to prevent the RDP from shutting down. When you need to restore or backup MC saves, stop this block, restore/backup, and start this block again. Note that RDP will always shut down after 12 hours, even if still being used.<br>

while True:pass

Thanks!